In [1]:
import pandas as pd

In [2]:
sequences = pd.read_csv('data/kit_sequenzen.csv', header=0)

In [5]:
df = pd.DataFrame(columns=['ID', 'sequence'])

In [4]:
# TODO
# generate legend for item translation
# remove double occurrences (only keep first)
# get item positions from annotation file
# define parameters for k

In [8]:
for row in range(0, len(sequences)):
    seq = []
    ID = sequences.at[row, 'ID']
    for item in sequences.at[row,'sequence'].split(','):
        if item not in seq:
            seq.append(item)
    df.at[row, 'ID'] = ID
    df.at[row, 'sequence'] = seq